# Using Pre-trained Word Embeddings

This notebook will demonstrate how to use pre-trained word embeddings.

To see why word embeddings are useful, it's worth comparing them to the alternative.
Without word embeddings, we might represent each word with a one-hot vector `[0, ...,0, 1, 0, ... 0]`, that takes value `1` at the index corresponding to the appropriate vocabulary word, 
and value `0` everywhere else. 
The weight matrices connecting our word-level inputs to the network's hidden layers would each be $v \times h$,
where $v$ is the size of the vocabulary and $h$ is the size of the hidden layer. 
With 100,000 words feeding into an LSTM layer with $1000$ nodes, the model would need to learn
$4$ different weight matrices (one for each of the LSTM gates), each with 100M weights, and thus 400 million parameters in total.

Fortunately, it turns out that a number of efficient techniques 
can quickly discover broadly useful word embeddings in an *unsupervised* manner.
These embeddings map each word onto a low-dimensional vector $w \in R^d$ with $d$ commonly chosen to be roughly $100$.
Intuitively, these embeddings are chosen based on the contexts in which words appear. 
Words that appear in similar contexts (like "tennis" and "racquet") should have similar embeddings
while words that do not like (like "rat" and "gourmet") should have dissimilar embeddings.

Practitioners of deep learning for NLP typically inititalize their models 
using *pretrained* word embeddings, bringing in outside information, and reducing the number of parameters that a neural network needs to learn from scratch.


Two popular word embeddings are Word2Vec and fastText. 
The following examples uses pre-trained word embeddings drawn from the following sources:

* Word2Vec https://arxiv.org/abs/1301.3781
* fastText project website：https://fasttext.cc/

To begin, let's first import a few packages that we'll need for this example:

In [1]:
import mxnet as mx
from mxnet import gluon, nd
from mxnet.gluon import rnn, nn
import gluonnlp as nlp
import re

import warnings
warnings.filterwarnings('ignore')

## Pre-trained Word Embeddings

GluonNLP provides a number of pre-trained Word Embeddings.

In [2]:
nlp.embedding.list_sources('fasttext')[:5]

['crawl-300d-2M', 'crawl-300d-2M-subword', 'wiki.aa', 'wiki.ab', 'wiki.ace']

For simplicity of demonstration, we use a smaller word embedding file, such as
the 50-dimensional one.

In [3]:
emb = nlp.embedding.create('fasttext', source='wiki.en')

### Word Similarity

Given an input word, we can find the nearest word from
the vocabulary by similarity. The
similarity between any pair of words can be represented by the cosine similarity
of their vectors.

In [4]:
def norm_vecs_by_row(x):
    return x / nd.sqrt(nd.sum(x * x, axis=1) + 1E-10).reshape((-1,1))

In [5]:
def get_knn(emb, k, word):
    word_vec = emb[word].reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(emb.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_vec)
    indices = nd.topk(dot_prod.reshape((len(emb.idx_to_token), )), k=k+1, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    return [emb.idx_to_token[i] for i in indices[1:]] # Remove input tokens.

Let us find the 5 most similar words of 'baby' from the vocabulary (size:
400,000 words).

In [6]:
get_knn(emb, 5, 'baby')

['babies', 'newborn', 'baby…', 'mom', 'toddler']

We can verify the cosine similarity of vectors of 'baby' and 'babies'.

In [7]:
from mxnet import nd
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

cos_sim(emb['baby'], emb['babies'])


[0.76009405]
<NDArray 1 @cpu(0)>

### Word Analogy

We can also apply pre-trained word embeddings to the word
analogy problem. 

For instance, "man : woman :: son : daughter" is an analogy.

The word analogy completion problem is defined as: for analogy 'a : b :: c : d',
given the first three words 'a', 'b', 'c', find 'd'. The idea is to find the
most similar word vector for vec('c') + (vec('b')-vec('a')).

In this example, we will find words by analogy from the 400,000 indexed words in `vocab`.

In [8]:
def get_top_k_by_analogy(emb, k, word1, word2, word3):
    word_vecs = emb[word1, word2, word3]
    word_diff = (word_vecs[1] - word_vecs[0] + word_vecs[2]).reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(emb.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_diff)
    indices = nd.topk(dot_prod.reshape((len(emb.idx_to_token), )), k=k, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    return [emb.idx_to_token[i] for i in indices]

Complete word analogy 'man : woman :: son :'.

In [9]:
get_top_k_by_analogy(emb, 1, 'man', 'woman', 'son')

['daughter']

# Text Classification and Data Sets

Text classification is a common task in natural language processing, which transforms a sequence of text of indefinite length into a category of text.

## Text Sentiment Classification Data

Use Stanford's Large Movie Review Dataset as the data set for text sentiment classification.
- Contains parts for training and testing purposes, each containing 25,000 movie reviews downloaded from IMDb
- In each data set, the number of comments labeled as "positive" and "negative" is equal.

## Dataset in `gluon`

Datasets in Gluon have the following basic structure:

``` python
class Dataset(object):
    def __getitem__(self, idx):
        ...
    
    def __len__(self):
        ...

    def transform(self, fn, lazy=True):
        # Returns a new dataset with each sample
        # transformed by the function `fn`.
```

We can make list-like Python object (ie. that implements `__getitem__` meaning it  subscripted like `x[0]` etc.),
into a `gluon` `Dataset` by wrapping it, using `gluon.data.SimpleDataset` as follows:

In [10]:
def tokenize_while_preserving_label(sample):
    sentence, label = sample
    return sentence.split(), label

train_dataset, test_dataset = [nlp.data.IMDB(segment=segment).transform(
    tokenize_while_preserving_label, lazy=False)
    for segment in ('train', 'test')]

In [11]:
print(train_dataset[0])

(['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"Teachers".', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', "High's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"Teachers".', 'The', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High.', 'A', 'classic', 'line:', 'INSPECTOR:', "I'm", 'here', 'to', 'sack', 'one', 'of', 'your', '

In [12]:
def get_text(text, label):
    return text
train_tokens = train_dataset.transform(get_text)

import itertools
vocab = nlp.Vocab(nlp.data.count_tokens(itertools.chain.from_iterable(train_tokens)), min_freq=10)
print(vocab)
print(vocab.idx_to_token[:10] + ["..."])

Vocab(size=28740, unk="<unk>", reserved="['<pad>', '<bos>', '<eos>']")
['<unk>', '<pad>', '<bos>', '<eos>', 'the', 'a', 'and', 'of', 'to', 'is', '...']


In [13]:
# `length_clip` takes as input a list and outputs a list with maximum length 500.
length_clip = nlp.data.ClipSequence(500)

def preprocess(data, label):
    label = int(label > 5)
    data = vocab[length_clip(data)]
    return [data, label]


train_dataset = train_dataset.transform(preprocess)
test_dataset = test_dataset.transform(preprocess)
print(train_dataset[0])

[[0, 2720, 9, 5, 1400, 1352, 58, 2225, 33, 4, 171, 85, 17, 48, 82, 9137, 44, 520, 778, 140, 17, 0, 361, 7185, 193, 10, 4, 5819, 8968, 495, 86, 8, 265, 12, 0, 0, 2638, 9, 80, 2646, 8, 899, 72, 9, 0, 21, 0, 8, 2718, 0, 4, 7999, 1898, 37, 66, 69, 259, 150, 57, 1710, 0, 0, 4, 0, 7, 4, 215, 5442, 39, 2989, 86, 7, 4, 8660, 11, 638, 6, 57, 9156, 285, 11, 209, 4, 484, 10, 64, 5, 1839, 4580, 765, 8, 4401, 217, 4, 3156, 11, 1371, 19648, 0, 33, 0, 27132, 135, 443, 7613, 0, 162, 227, 8, 14323, 34, 7, 119, 0, 0, 18376, 8, 0, 27132, 11, 546, 12, 104, 2044, 7, 68, 879, 100, 12, 0, 2720, 9, 239, 0, 257, 5, 3155, 12, 14, 0], 1]


#### `gluonnlp.data.batchify.Pad`

`gluonnlp.data.batchify.Pad` can be instantiated with the padding value.
The resulting function takes a list of variable length lists (or NDArrays or numpy arrays)
as input and returns a single NDArray where all shorter inputs are padded to the length
of the maximum length input element.

In [14]:
print(len(train_dataset[0][0]), len(train_dataset[1][0]))

140 428


In [15]:
# Pad data, stack label and lengths
pad_val = vocab[vocab.padding_token]
batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(axis=0, ret_length=True, pad_val=pad_val),
    nlp.data.batchify.Stack(dtype='float32'))
print('pad_val:', pad_val)

pad_val: 1


In [16]:
batchify_fn([train_dataset[0], train_dataset[1], train_dataset[2]])

((
  [[0.0000e+00 2.7200e+03 9.0000e+00 ... 1.0000e+00 1.0000e+00 1.0000e+00]
   [0.0000e+00 9.0800e+02 0.0000e+00 ... 8.0000e+00 3.3700e+02 2.2630e+03]
   [1.1217e+04 1.7181e+04 3.2000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]]
  <NDArray 3x428 @cpu_shared(0)>, 
  [140 428 147]
  <NDArray 3 @cpu_shared(0)>), 
 [1. 1. 1.]
 <NDArray 3 @cpu_shared(0)>)

#### `gluon.data.DataLoader`

Manually sampling sentences from the dataset and applying the pad function may be bothersome.
`gluon.data.DataLoader` automates this process.

In [17]:
print(gluon.data.DataLoader.__doc__)

Loads data from a dataset and returns mini-batches of data.

    Parameters
    ----------
    dataset : Dataset
        Source dataset. Note that numpy and mxnet arrays can be directly used
        as a Dataset.
    batch_size : int
        Size of mini-batch.
    shuffle : bool
        Whether to shuffle the samples.
    sampler : Sampler
        The sampler to use. Either specify sampler or shuffle, not both.
    last_batch : {'keep', 'discard', 'rollover'}
        How to handle the last batch if batch_size does not evenly divide
        `len(dataset)`.

        keep - A batch with less samples than previous batches is returned.
        discard - The last batch is discarded if its incomplete.
        rollover - The remaining samples are rolled over to the next epoch.
    batch_sampler : Sampler
        A sampler that returns mini-batches. Do not specify batch_size,
        shuffle, sampler, and last_batch if batch_sampler is specified.
    batchify_fn : callable
        Callback fun

In [18]:
batch_size = 64
data_loader = gluon.data.DataLoader(train_dataset, batchify_fn=batchify_fn,
                                    batch_size=batch_size)
print('Average length of batches is', sum(batch[0][0].shape[1] for batch in data_loader) / len(data_loader))

Average length of batches is 499.5089514066496


<img src="../img/fixed_bucket_strategy_ratio0.7.png" style="width: 100%;"/>

##### Custom `Sampler` for `DataLoader`

Sampling random sentences from the Dataset and padding them is sub-optimal as the number of padding elments is determined by the longest sentence.
`DataLoader` supports the specification of a `Sampler` to specify the sentences to select for a batch.

For example, `gluonnlp.data.FixedBucketSampler` assigns each data sample (sentence)
to a fixed bucket based on its length. Resulting batches will only contain sentences
from a single bucket.

This can significantly reduce the average number of elements per batch and consequently the amount of computation.

In [19]:
print(nlp.data.FixedBucketSampler.__doc__)

Assign each data sample to a fixed bucket based on its length.
    The bucket keys are either given or generated from the input sequence lengths.

    Parameters
    ----------
    lengths : list of int or list of tuple/list of int
        The length of the sequences in the input data sample.
    batch_size : int
        The batch size of the sampler.
    num_buckets : int or None, default 10
        The number of buckets. This will not be used if bucket_keys is set.
    bucket_keys : None or list of int or list of tuple, default None
        The keys that will be used to create the buckets. It should usually be the lengths of the
        sequences. If it is None, the bucket_keys will be generated based on the maximum
        lengths of the data.
    ratio : float, default 0
        Ratio to scale up the batch size of smaller buckets.
        Assume the :math:`i` th key is :math:`K_i` ,
        the default batch size is :math:`B` , the ratio to scale the batch size is
        :math:`\a

In [20]:
train_sampler = nlp.data.FixedBucketSampler(
    lengths=train_dataset.transform(lambda x: len(x[0]), lazy=False),
    batch_size=batch_size, shuffle=True)
test_sampler = nlp.data.FixedBucketSampler(
    lengths=test_dataset.transform(lambda x: len(x[0]), lazy=False),
    batch_size=batch_size, shuffle=True)

print(train_sampler.stats())

FixedBucketSampler:
  sample_num=25000, batch_num=396
  key=[59, 108, 157, 206, 255, 304, 353, 402, 451, 500]
  cnt=[1036, 2543, 7173, 4296, 2619, 1729, 1330, 954, 733, 2587]
  batch_size=[64, 64, 64, 64, 64, 64, 64, 64, 64, 64]


In [21]:
train_dataloader = gluon.data.DataLoader(train_dataset, batchify_fn=batchify_fn, batch_sampler=train_sampler)
test_dataloader = gluon.data.DataLoader(test_dataset, batchify_fn=batchify_fn, batch_sampler=test_sampler)
print('Average length of batches is',
      sum(batch[0][0].shape[1] for batch in train_dataloader) / len(train_dataloader))

Average length of batches is 240.4090909090909


In [22]:
next(iter(train_dataloader))

((
  [[5.100e+01 5.200e+01 2.700e+01 ... 1.000e+00 1.000e+00 1.000e+00]
   [1.575e+03 4.000e+01 1.320e+02 ... 1.000e+00 1.000e+00 1.000e+00]
   [5.100e+01 2.600e+01 9.000e+00 ... 1.000e+00 1.000e+00 1.000e+00]
   ...
   [1.100e+01 1.057e+03 1.800e+01 ... 1.000e+00 1.000e+00 1.000e+00]
   [1.350e+02 8.430e+02 2.600e+01 ... 1.000e+00 1.000e+00 1.000e+00]
   [1.100e+01 2.860e+02 1.300e+01 ... 1.000e+00 1.000e+00 1.000e+00]]
  <NDArray 64x156 @cpu_shared(0)>, 
  [128 138 134 141 147 144 113 154 156 130 126 124 120 151 126 150 120 152
   113 133 144 111 147 127 138 131 133 115 129 113 118 131 132 149 145 116
   137 113 117 128 120 132 137 110 116 112 142 128 133 120 144 152 148 122
   141 124 136 110 117 136 142 119 138 109]
  <NDArray 64 @cpu_shared(0)>), 
 [0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1.
  1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
  1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0.]
 <NDArray 64 @cpu_shared(0)>)

# Classification Models: Using a Bag of Context Free Embeddings

In [23]:
class ContinuousBagOfWords(gluon.Block):
    def __init__(self, vocab_size, embed_size, **kwargs):
        super(ContinuousBagOfWords, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.decoder = nn.Dense(2)

    def forward(self, inputs):
        # The shape of inputs is (batch size, number of words).
        embeddings = self.embedding(inputs)
        encoding = embeddings.mean(axis=1)
        outputs = self.decoder(encoding)
        return outputs

In [24]:
embed_size, ctx = 300, mx.gpu(0)
net = ContinuousBagOfWords(len(vocab), embed_size)
net.initialize(mx.init.Xavier(), ctx=ctx)

Next we load the pre-trained word vectors,

In [25]:
emb = nlp.embedding.create('fasttext', source='wiki.simple', load_ngrams=True)
idx_to_vec = emb[vocab.idx_to_token]
idx_to_vec.shape

Embedding file wiki.simple.npz is not found. Downloading from Gluon Repository. This may take some time.
Embedding file wiki.simple-67424112.bin is not found. Downloading from Gluon Repository. This may take some time.


(28740, 300)

and use these word vectors as feature vectors for each word in the reviews. 

In [26]:
net.embedding.weight.set_data(idx_to_vec)
net.embedding.collect_params().setattr('grad_req', 'null')

### Train and Evaluate the Model



In [27]:
def train(net, train_iter, test_iter, loss, trainer, num_epochs, ctx):
    num_batches = len(train_iter)
    for epoch in range(num_epochs):
        metric = [0.0] * 4
        for i, ((features, lengths), labels) in enumerate(train_iter):
            features = features.as_in_context(ctx)
            labels = labels.as_in_context(ctx)
            with mx.autograd.record():
                y = net(features)
                l = loss(y, labels)
            l.backward()
            trainer.step(features.shape[0])

            acc = (y.argmax(axis=1) == labels).sum().asscalar()
            
            metric = [a+b for a, b in zip(metric, (l.sum().asscalar(), acc, labels.shape[0], labels.size))]
            if (i+1) % (num_batches // 5) == 0:
                print('loss %.3f, train acc %.3f' % (metric[0]/metric[2], metric[1]/metric[3]))
                
                
        # Evaluate
        test_acc = 0
        for i, ((features, lengths), labels) in enumerate(test_iter):
            y = net(features.as_in_context(ctx))
            test_acc += (y.argmax(axis=1) == labels.as_in_context(ctx)).mean().asscalar()
        test_acc /= len(test_iter)

        print('loss %.3f, train acc %.3f, test acc %.3f' % (
            metric[0]/metric[2], metric[1]/metric[3], test_acc))

In [28]:
lr, num_epochs = 0.01, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train(net, train_dataloader, test_dataloader, loss, trainer, num_epochs, ctx)

loss 0.675, train acc 0.574
loss 0.654, train acc 0.614
loss 0.638, train acc 0.639
loss 0.625, train acc 0.658
loss 0.617, train acc 0.666
loss 0.617, train acc 0.667, test acc 0.733
loss 0.564, train acc 0.724
loss 0.561, train acc 0.727
loss 0.555, train acc 0.735
loss 0.546, train acc 0.743
loss 0.543, train acc 0.743
loss 0.543, train acc 0.743, test acc 0.714
loss 0.521, train acc 0.763
loss 0.515, train acc 0.766
loss 0.519, train acc 0.759
loss 0.515, train acc 0.762
loss 0.512, train acc 0.764
loss 0.512, train acc 0.764, test acc 0.764
loss 0.494, train acc 0.775
loss 0.492, train acc 0.776
loss 0.493, train acc 0.775
loss 0.491, train acc 0.776
loss 0.491, train acc 0.777
loss 0.491, train acc 0.777, test acc 0.776
loss 0.487, train acc 0.773
loss 0.484, train acc 0.777
loss 0.481, train acc 0.780
loss 0.480, train acc 0.780
loss 0.478, train acc 0.781
loss 0.478, train acc 0.781, test acc 0.768


We define a prediction function:

In [29]:
def predict_sentiment(net, vocab, sentence):
    sentence = nd.array(vocab[sentence.split()], ctx=ctx)
    label = nd.argmax(net(sentence.reshape((1, -1))), axis=1)
    return 'positive' if label.asscalar() == 1 else 'negative'

In [30]:
predict_sentiment(net, vocab, 'this movie is so great')

'positive'

In [31]:
predict_sentiment(net, vocab, 'this movie is so bad')

'negative'

# Using Recurrent Neural Networks

In this section, we will apply
pre-trained word vectors and bidirectional recurrent neural networks with
multiple hidden layers:

Maas, Andrew L., et al. "Learning word vectors for sentiment analysis." Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies-volume 1. Association for Computational Linguistics, 2011.

## Bidirectional RNNs

![BiRNN](../img/birnn.svg)


## Use a Recurrent Neural Network Model

In this model, each word first obtains a feature vector from the embedding
layer. Then, we further encode the feature sequence using a bidirectional
recurrent neural network to obtain sequence information. Finally, we transform
the encoded sequence information to output through the fully connected
layer. Specifically, we can concatenate hidden states of bidirectional
long-short term memory in the initial time step and final time step and pass it
to the output layer classification as encoded feature sequence information. In
the `BiRNN` class implemented below, the `Embedding` instance is the embedding
layer, the `LSTM` instance is the hidden layer for sequence encoding, and the
`Dense` instance is the output layer for generated classification results.

In [32]:
class BiRNN(nn.Block):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Set Bidirectional to True to get a bidirectional recurrent neural
        # network
        self.encoder = rnn.LSTM(num_hiddens, num_layers=num_layers,
                                bidirectional=True, input_size=embed_size)
        self.decoder = nn.Dense(2)

    def forward(self, inputs):
        # The shape of inputs is (batch size, number of words). Because LSTM
        # needs to use sequence as the first dimension, the input is
        # transformed and the word feature is then extracted. The output shape
        # is (number of words, batch size, word vector dimension).
        embeddings = self.embedding(mx.nd.transpose(inputs))
        # Since the input (embeddings) is the only argument passed into
        # rnn.LSTM, it only returns the hidden states of the last hidden layer
        # at different time step (outputs). The shape of outputs is
        # (number of words, batch size, 2 * number of hidden units).
        outputs = self.encoder(embeddings)
        # Concatenate the hidden states of the initial time step and final
        # time step to use as the input of the fully connected layer. Its
        # shape is (batch size, 4 * number of hidden units)
        encoding = mx.nd.concat(outputs[0], outputs[-1])
        outs = self.decoder(encoding)
        return outs

Create a bidirectional recurrent neural network with two hidden layers.

In [33]:
num_hiddens, num_layers = 100, 2
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)
net.initialize(mx.init.Xavier(), ctx=ctx)

### Load Pre-trained Word Vectors

Because the training data set for sentiment classification is not very large, in order to deal with overfitting, we will directly use word vectors pre-trained on a larger corpus as the feature vectors of all words. Here, we load a 100-dimensional GloVe word vector for each word in the dictionary `vocab`.

In [34]:
net.embedding.weight.set_data(idx_to_vec)
net.embedding.collect_params().setattr('grad_req', 'null')

### Train and Evaluate the Model

Now, we can start training.

In [35]:
lr, num_epochs = 0.01, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train(net, train_dataloader, test_dataloader, loss, trainer, num_epochs, ctx)

loss 0.700, train acc 0.512
loss 0.686, train acc 0.544
loss 0.637, train acc 0.609
loss 0.597, train acc 0.654
loss 0.565, train acc 0.684
loss 0.564, train acc 0.685, test acc 0.827
loss 0.372, train acc 0.841
loss 0.367, train acc 0.841
loss 0.401, train acc 0.821
loss 0.398, train acc 0.823
loss 0.401, train acc 0.822
loss 0.400, train acc 0.822, test acc 0.838
loss 0.351, train acc 0.852
loss 0.345, train acc 0.852
loss 0.343, train acc 0.853
loss 0.338, train acc 0.855
loss 0.337, train acc 0.855
loss 0.337, train acc 0.855, test acc 0.857
loss 0.299, train acc 0.870
loss 0.296, train acc 0.875
loss 0.299, train acc 0.873
loss 0.306, train acc 0.870
loss 0.307, train acc 0.870
loss 0.307, train acc 0.870, test acc 0.863
loss 0.251, train acc 0.898
loss 0.260, train acc 0.894
loss 0.269, train acc 0.890
loss 0.275, train acc 0.886
loss 0.276, train acc 0.886
loss 0.276, train acc 0.886, test acc 0.862


In [36]:
predict_sentiment(net, vocab, 'this movie is so great')

'negative'

In [37]:
predict_sentiment(net, vocab, 'this movie is so bad')

'negative'

# Using Convolutional Neural Networks (textCNN)

Idea: treat text as a one-dimensional "image".

Then we can use one-dimensional convolutional neural networks to capture associations between adjacent words.

This section describes a groundbreaking approach to applying
convolutional neural networks to text analysis: textCNN :cite:`Kim.2014`.

## One-dimensional Convolutional Layer

![One-dimensional cross-correlation operation. The shaded parts are the first output element as well as the input and kernel array elements used in its calculation: $0\times1+1\times2=2$. ](../img/conv1d.svg)

Before introducing the model, let us explain how a one-dimensional convolutional layer works. Like a two-dimensional convolutional layer, a one-dimensional convolutional layer uses a one-dimensional cross-correlation operation. In the one-dimensional cross-correlation operation, the convolution window starts from the leftmost side of the input array and slides on the input array from left to right successively. When the convolution window slides to a certain position, the input subarray in the window and kernel array are multiplied and summed by element to get the element at the corresponding location in the output array. As shown in Figure 12.4, the input is a one-dimensional array with a width of 7 and the width of the kernel array is 2. As we can see, the output width is $7-2+1=6$ and the first element is obtained by performing multiplication by element on the leftmost input subarray with a width of 2 and kernel array and then summing the results.


Next, we implement one-dimensional cross-correlation in the `corr1d` function. It accepts the input array `X` and kernel array `K` and outputs the array `Y`.

In [38]:
def corr1d(X, K):
    w = K.shape[0]
    Y = nd.zeros((X.shape[0] - w + 1))
    for i in range(Y.shape[0]):
        Y[i] = (X[i: i + w] * K).sum()
    return Y

Let's reproduce the results of the one-dimensional cross-correlation operation seen in above Figure.

In [39]:
X, K = nd.array([0, 1, 2, 3, 4, 5, 6]), nd.array([1, 2])
corr1d(X, K)


[ 2.  5.  8. 11. 14. 17.]
<NDArray 6 @cpu(0)>

## One-dimensional Convolutional Layer with multiple input channels


![One-dimensional cross-correlation operation with three input channels. The shaded parts are the first output element as well as the input and kernel array elements used in its calculation: $0\times1+1\times2+1\times3+2\times4+2\times(-1)+3\times(-3)=2$. ](../img/conv1d-channel.svg)

The one-dimensional cross-correlation operation for multiple input channels is also similar to the two-dimensional cross-correlation operation for multiple input channels. On each channel, it performs the one-dimensional cross-correlation operation on the kernel and its corresponding input and adds the results of the channels to get the output. Figure 12.5 shows a one-dimensional cross-correlation operation with three input channels.

Now, we reproduce the results of the one-dimensional cross-correlation operation with multi-input channel.

In [40]:
def corr1d_multi_in(X, K):
    return nd.add_n(*[corr1d(x, k) for x, k in zip(X, K)])

In [41]:
X = nd.array([[0, 1, 2, 3, 4, 5, 6],
              [1, 2, 3, 4, 5, 6, 7],
              [2, 3, 4, 5, 6, 7, 8]])
K = nd.array([[1, 2], [3, 4], [-1, -3]])
corr1d_multi_in(X, K)


[ 2.  8. 14. 20. 26. 32.]
<NDArray 6 @cpu(0)>

This is equivalent to two-dimensional cross-correlation with a single input channel

![Two-dimensional cross-correlation operation with a single input channel. The highlighted parts are the first output element and the input and kernel array elements used in its calculation: $2\times(-1)+3\times(-3)+1\times3+2\times4+0\times1+1\times2=2$. ](../img/conv1d-2d.svg)

We can obtain multiple output channels by applying the cross-correlation multiple times with different kernels.

## Max-Over-Time Pooling Layer

Similarly, we have a one-dimensional pooling layer. The max-over-time pooling layer used in TextCNN actually corresponds to a one-dimensional global maximum pooling layer. Assuming that the input contains multiple channels, and each channel consists of values on different time steps, the output of each channel will be the largest value of all time steps in the channel. Therefore, the input of the max-over-time pooling layer can have different time steps on each channel.

To improve computing performance, we often combine timing examples of different lengths into a mini-batch and make the lengths of each timing example in the batch consistent by appending special characters (such as 0) to the end of shorter examples. Naturally, the added special characters have no intrinsic meaning. Because the main purpose of the max-over-time pooling layer is to capture the most important features of timing, it usually allows the model to be unaffected by the manually added characters.

## The TextCNN Model

![TextCNN design. ](../img/textcnn.svg)

Kim, Yoon. "Convolutional neural networks for sentence classification." EMNLP 2014.

TextCNN mainly uses a one-dimensional convolutional layer and max-over-time pooling layer. Suppose the input text sequence consists of $n$ words, and each word is represented by a $d$-dimension word vector. Then the input example has a width of $n$, a height of 1, and $d$ input channels. The calculation of textCNN can be mainly divided into the following steps:

1. Define multiple one-dimensional convolution kernels and use them to perform convolution calculations on the inputs. Convolution kernels with different widths may capture the correlation of different numbers of adjacent words.
2. Perform max-over-time pooling on all output channels, and then concatenate the pooling output values of these channels in a vector.
3. The concatenated vector is transformed into the output for each category through the fully connected layer. A dropout layer can be used in this step to deal with overfitting.

Figure 12.7 gives an example to illustrate the textCNN. The input here is a sentence with 11 words, with each word represented by a 6-dimensional word vector. Therefore, the input sequence has a width of 11 and 6 input channels. We assume there are two one-dimensional convolution kernels with widths of 2 and 4, and 4 and 5 output channels, respectively. Therefore, after one-dimensional convolution calculation, the width of the four output channels is $11-2+1=10$, while the width of the other five channels is $11-4+1=8$. Even though the width of each channel is different, we can still perform max-over-time pooling for each channel and concatenate the pooling outputs of the 9 channels into a 9-dimensional vector. Finally, we use a fully connected layer to transform the 9-dimensional vector into a 2-dimensional output: positive sentiment and negative sentiment predictions.

Next, we will implement a textCNN model. Compared with the previous section, in addition to replacing the recurrent neural network with a one-dimensional convolutional layer, here we use two embedding layers, one with a fixed weight and another that participates in training.

In [42]:
class TextCNN(nn.Block):
    def __init__(self, vocab_size, embed_size, kernel_sizes, num_channels,
                 **kwargs):
        super(TextCNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # The embedding layer does not participate in training
        self.constant_embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Dense(2)
        # The max-over-time pooling layer has no weight, so it can share an
        # instance
        self.pool = nn.GlobalMaxPool1D()
        # Create multiple one-dimensional convolutional layers
        self.convs = nn.Sequential()
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.add(nn.Conv1D(c, k, activation='relu'))

    def forward(self, inputs):
        # Concatenate the output of two embedding layers with shape of
        # (batch size, number of words, word vector dimension) by word vector
        embeddings = nd.concat(
            self.embedding(inputs), self.constant_embedding(inputs), dim=2)
        # According to the input format required by Conv1D, the word vector
        # dimension, that is, the channel dimension of the one-dimensional
        # convolutional layer, is transformed into the previous dimension
        embeddings = embeddings.transpose((0, 2, 1))
        # For each one-dimensional convolutional layer, after max-over-time
        # pooling, an NDArray with the shape of (batch size, channel size, 1)
        # can be obtained. Use the flatten function to remove the last
        # dimension and then concatenate on the channel dimension
        encoding = nd.concat(*[nd.flatten(
            self.pool(conv(embeddings))) for conv in self.convs], dim=1)
        # After applying the dropout method, use a fully connected layer to
        # obtain the output
        outputs = self.decoder(self.dropout(encoding))
        return outputs

Create a TextCNN instance. It has 3 convolutional layers with kernel widths of 3, 4, and 5, all with 100 output channels.

In [43]:
kernel_sizes, nums_channels = [3, 4, 5], [100, 100, 100]
net = TextCNN(len(vocab), embed_size, kernel_sizes, nums_channels)
net.initialize(mx.init.Xavier(), ctx=ctx)

### Load Pre-trained Word Vectors

As in the previous section, load pre-trained 100-dimensional GloVe word vectors and initialize the embedding layers `embedding` and `constant_embedding`. Here, the former participates in training while the latter has a fixed weight.

In [44]:
net.embedding.weight.set_data(idx_to_vec)
net.constant_embedding.weight.set_data(idx_to_vec)
net.constant_embedding.collect_params().setattr('grad_req', 'null')

### Train and Evaluate the Model

Now we can train the model.

In [45]:
lr, num_epochs = 0.001, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train(net, train_dataloader, test_dataloader, loss, trainer, num_epochs, ctx)

loss 0.671, train acc 0.644
loss 0.574, train acc 0.706
loss 0.534, train acc 0.732
loss 0.501, train acc 0.754
loss 0.480, train acc 0.768
loss 0.480, train acc 0.768, test acc 0.828
loss 0.303, train acc 0.871
loss 0.306, train acc 0.869
loss 0.308, train acc 0.870
loss 0.302, train acc 0.873
loss 0.297, train acc 0.875
loss 0.297, train acc 0.875, test acc 0.874
loss 0.189, train acc 0.928
loss 0.191, train acc 0.926
loss 0.209, train acc 0.918
loss 0.208, train acc 0.917
loss 0.207, train acc 0.917
loss 0.207, train acc 0.917, test acc 0.893
loss 0.113, train acc 0.957
loss 0.113, train acc 0.958
loss 0.117, train acc 0.956
loss 0.121, train acc 0.955
loss 0.121, train acc 0.954
loss 0.121, train acc 0.954, test acc 0.892
loss 0.067, train acc 0.979
loss 0.065, train acc 0.978
loss 0.066, train acc 0.979
loss 0.065, train acc 0.978
loss 0.069, train acc 0.976
loss 0.069, train acc 0.976, test acc 0.884


Below, we use the trained model to the classify sentiments of two simple sentences.

In [46]:
predict_sentiment(net, vocab, 'this movie is so great')

'negative'

In [50]:
predict_sentiment(net, vocab, 'this movie is so bad')

'negative'